In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [9]:
path = '../raw_data/train'

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=90,
    validation_split = 0.2)

In [11]:
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2)

In [12]:
train_generator = train_datagen.flow_from_directory(
    path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed = 123)

Found 3460 images belonging to 16 classes.


In [13]:
val_generator = valid_datagen.flow_from_directory(
    path, # same directory as training data
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed = 123)

Found 857 images belonging to 16 classes.


In [14]:
vgg16_model = VGG16(include_top=False, input_shape=(256,256,3))

model=Sequential()
for layers in vgg16_model.layers:
    model.add(layers)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
for layer in model.layers:
    layer.trainable=False
model.add(Dense(16,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 64, 64, 256)      

In [ ]:
# # Set the first layers to be untrainable
# model.trainable = False
# # Add layers to the mdoel
# flatten_layer = layers.Flatten()
# dense_layer = layers.Dense(100, activation='relu')
# prediction_layer = layers.Dense(16, activation='softmax')

# model = Sequential([model,
#                     flatten_layer,
#                     dense_layer,
#                     prediction_layer])
# model.summary()

In [24]:
# model.compile(optimizer=optimizers.SGD(learning_rate = 0.1),
#           loss=SparseCategoricalCrossentropy(from_logits=True),
#           metrics=['accuracy'])

In [15]:
optimizer=RMSprop(lr=0.001,rho=0.9,epsilon=1e-08,decay=0.0)

In [16]:
model.compile(optimizer,loss=SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [17]:
learning_rate_reduction=ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.5,minlr=0.00001)

In [22]:
# callbacks = EarlyStopping(
#     monitor='val_loss',
#     patience=3,
#     mode='min')

In [19]:
history=model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = val_generator,
    validation_steps = val_generator.samples // 32,
    epochs = 30,
    callbacks = [learning_rate_reduction])

Epoch 1/30
108/108 [==============================] - 880s 8s/step - loss: 2.6897 - accuracy: 0.1820 - val_loss: 2.6611 - val_accuracy: 0.2320
Epoch 2/30
108/108 [==============================] - 616s 6s/step - loss: 2.6377 - accuracy: 0.2462 - val_loss: 2.6195 - val_accuracy: 0.2812
Epoch 3/30
108/108 [==============================] - 628s 6s/step - loss: 2.6023 - accuracy: 0.2943 - val_loss: 2.5673 - val_accuracy: 0.3389
Epoch 4/30
108/108 [==============================] - 633s 6s/step - loss: 2.5747 - accuracy: 0.3264 - val_loss: 2.5546 - val_accuracy: 0.3401
Epoch 5/30
108/108 [==============================] - 648s 6s/step - loss: 2.5563 - accuracy: 0.3431 - val_loss: 2.5360 - val_accuracy: 0.3642
Epoch 6/30
108/108 [==============================] - 694s 6s/step - loss: 2.5432 - accuracy: 0.3539 - val_loss: 2.5302 - val_accuracy: 0.3582
Epoch 7/30
108/108 [==============================] - 672s 6s/step - loss: 2.5394 - accuracy: 0.3562 - val_loss: 2.5209 - val_accuracy: 0.3798

In [35]:
model.save(r'../model/roxane_vgg_2')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../model/roxane_vgg_1/assets
